# Tagging and Extraction Using OpenAI functions

In [1]:
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting
####.  Please use VirttualEnv: LCEL_extracting

import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 
OPENAI_API_KEY=os.environ["OPENAI_API_KEY"]


In [2]:
!pip freeze | grep langchain

!pip freeze | grep openai

!pip freeze | grep pydantic


langchain==0.2.15
langchain-community==0.2.13
langchain-core==0.2.41
langchain-openai==0.1.23
langchain-text-splitters==0.2.4
langchain-openai==0.1.23
openai==1.47.0
openapi-schema-pydantic==1.2.4
pydantic==2.9.2
pydantic-settings==2.5.2
pydantic_core==2.23.4


## Extraction

Extraction is similar to tagging, but used for extracting multiple pieces of information.

In [3]:
from langchain_openai import ChatOpenAI
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.utils.openai_functions import convert_pydantic_to_openai_function
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser

model = ChatOpenAI(temperature=0,model="gpt-4o")

In [4]:
#from typing import Optional
class Person(BaseModel):
    """Information about a person."""
    name: str = Field(description="person's name")
    age: Optional[int] = Field(description="person's age")

In [5]:
class Information(BaseModel):
    """Information to extract."""
    people: List[Person] = Field(description="List of info about people")

In [6]:
convert_to_openai_function(Information)

{'name': 'Information',
 'description': 'Information to extract.',
 'parameters': {'$defs': {'Person': {'description': 'Information about a person.',
    'properties': {'name': {'description': "person's name", 'type': 'string'},
     'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
      'description': "person's age"}},
    'required': ['name', 'age'],
    'type': 'object'}},
  'properties': {'people': {'description': 'List of info about people',
    'items': {'description': 'Information about a person.',
     'properties': {'name': {'description': "person's name", 'type': 'string'},
      'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
       'description': "person's age"}},
     'required': ['name', 'age'],
     'type': 'object'},
    'type': 'array'}},
  'required': ['people'],
  'type': 'object'}}

In [7]:
#extraction_functions = [convert_pydantic_to_openai_function(Information)]
extraction_functions = [convert_to_openai_function(Information)]
extraction_model = model.bind(functions=extraction_functions, function_call={"name":"Information"})
extraction_model.invoke("Joe is 30. Joe's mom is Martha")

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"people":[{"name":"Joe","age":30},{"name":"Martha","age":null}]}', 'name': 'Information'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 90, 'total_tokens': 112, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_c17d3befe7', 'finish_reason': 'stop', 'logprobs': None}, id='run-ec725c2a-1da5-4e69-b844-063dc13c1723-0', usage_metadata={'input_tokens': 90, 'output_tokens': 22, 'total_tokens': 112})

Similarly we can use a separate output parser to pluck that "Information" key, since that's the information we really care about

In [8]:
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
extraction_chain = extraction_model | JsonKeyOutputFunctionsParser(key_name="people")
extraction_chain.invoke("Joe is 30. Joe's mom is Martha")

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]

In [9]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "Extract the relevant information, if not explicitly provided do not guess. Extract partial info"),
    ("human", "{input}")
])

In [10]:
extraction_chain = prompt | extraction_model | JsonKeyOutputFunctionsParser(key_name="people")

In [11]:
extraction_chain.invoke({"input": "Joe is 30. Joe's mom is Martha"})

[{'name': 'Joe', 'age': 30}, {'name': 'Martha', 'age': None}]